# HDC model training notebook 

Steps:
- load raw data 
- generate configs (hyper parameter)
- do cross validation to find the best configs (hyper parameters)
- train the model with the best hyperparameters in entire data from cross validation step 
- evaluate the model with test data (not included in cross validation train/val set)
- save the model and result

Note:
in this notebook, the final performance of the model is evaluated with test data.


In [34]:
MODEL_NAME='HDC_statistical_v2'

In [35]:
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold
from tqdm import tqdm
from scipy import stats
from scipy.special import softmax
import random
import builtins
import torch
import torch.nn as nn
import torch.nn.functional as F
#Use local Executorch compatible copy of TorchHD
import os
sys.path.insert(0, os.path.abspath("../../../torchhd"))
sys.path.insert(0, os.path.abspath("../../../torchhd/torchhd"))
import torchhd
from torchhd import embeddings
from torchhd import models
print(torchhd.__file__) #Check
print(embeddings.__file__) #Check
print(models.__file__) #Check
from typing import Union, Literal
import json 
import pickle
# import torchmetrics
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import (
    f1_score,
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
)
from sklearn.metrics import (
    mean_squared_error,
    median_absolute_error,
    r2_score,
    average_precision_score,
)
import warnings
from sklearn.preprocessing import StandardScaler
import gc
import time
from datetime import datetime
import logging
from tqdm import tqdm
from IPython.display import display
import os
from glob import glob
import polars as pl 
warnings.filterwarnings("ignore")


random.seed(0)
torch.manual_seed(0)
np.random.seed(0)

In [36]:
import sys, os

# Clear any cached torchhd (in case it was already imported)
sys.modules.pop("torchhd", None)

# Insert path to your local torchhd *before* importing it
sys.path.insert(0, os.path.abspath("/Users/jofremosegui/Desktop/TFG/wearbac_experiments/torchhd"))
import torchhd

In [37]:
hasattr(models.Centroid, "add_adjust")

True

In [38]:
# === Feature Columns for Statistical Data ===
STAT_FEATURE_COLUMNS = [
    # Accelerometer stats
    "ZVALUEX_acc_mean", "ZVALUEX_acc_std", "ZVALUEX_acc_min", "ZVALUEX_acc_max", "ZVALUEX_acc_median", "ZVALUEX_acc_range", "ZVALUEX_acc_iqr", "ZVALUEX_acc_skew", "ZVALUEX_acc_kurtosis",
    "ZVALUEY_acc_mean", "ZVALUEY_acc_std", "ZVALUEY_acc_min", "ZVALUEY_acc_max", "ZVALUEY_acc_median", "ZVALUEY_acc_range", "ZVALUEY_acc_iqr", "ZVALUEY_acc_skew", "ZVALUEY_acc_kurtosis",
    "ZVALUEZ_acc_mean", "ZVALUEZ_acc_std", "ZVALUEZ_acc_min", "ZVALUEZ_acc_max", "ZVALUEZ_acc_median", "ZVALUEZ_acc_range", "ZVALUEZ_acc_iqr", "ZVALUEZ_acc_skew", "ZVALUEZ_acc_kurtosis",
    
    # Gyroscope stats
    "ZVALUEX_gyro_mean", "ZVALUEX_gyro_std", "ZVALUEX_gyro_min", "ZVALUEX_gyro_max", "ZVALUEX_gyro_median", "ZVALUEX_gyro_range", "ZVALUEX_gyro_iqr", "ZVALUEX_gyro_skew", "ZVALUEX_gyro_kurtosis",
    "ZVALUEY_gyro_mean", "ZVALUEY_gyro_std", "ZVALUEY_gyro_min", "ZVALUEY_gyro_max", "ZVALUEY_gyro_median", "ZVALUEY_gyro_range", "ZVALUEY_gyro_iqr", "ZVALUEY_gyro_skew", "ZVALUEY_gyro_kurtosis",
    "ZVALUEZ_gyro_mean", "ZVALUEZ_gyro_std", "ZVALUEZ_gyro_min", "ZVALUEZ_gyro_max", "ZVALUEZ_gyro_median", "ZVALUEZ_gyro_range", "ZVALUEZ_gyro_iqr", "ZVALUEZ_gyro_skew", "ZVALUEZ_gyro_kurtosis",
    
    # Heart rate stats
    "ZHEARTRATE_mean", "ZHEARTRATE_std", "ZHEARTRATE_min", "ZHEARTRATE_max", "ZHEARTRATE_median", "ZHEARTRATE_range", "ZHEARTRATE_iqr", "ZHEARTRATE_skew", "ZHEARTRATE_kurtosis"
]

# === All columns to load from CSV ===
RAW_COLUMNS = ['user_id', 'session_id', 'tac_flg'] + STAT_FEATURE_COLUMNS

# === Threshold and user info ===
TAC_THRESHOLD = 35
TAC_LEVEL_0 = 0
TAC_LEVEL_1 = 1
NUM_TAC_LEVELS = 2

ALL_USERS = [6, 9, 10, 11, 14, 15, 16, 24, 25, 26, 28, 31]

TRAIN_USERS = [
    [9, 10, 14, 15, 24, 28, 31],
    [10, 11, 6, 31],
    [6, 9, 11, 14, 15, 24, 28]
]

VALID_USERS = [
    [11, 6],
    [9, 14, 15, 24, 28],
    [10, 31]
]

TEST_USERS = [16, 25, 26]

# === Base configuration for HDC model ===
BASE_CONFIGS = {
    "device": "mps" if torch.backends.mps.is_available() else "cpu",
    "window_size": [1],  # Ignored in statistical input mode
    "ngrams": [7],
    "hdc_dimension": 5000,
    "batch_size": [64],
    "learning_rate": [2],
    "epochs": 10,
    "patience": 5,
    "overlap_ratio": 0.5,
}


In [ ]:
def load_data(preprocess_fld=''):
    file_paths = sorted(glob(preprocess_fld + './features_statistical_group*.csv'))
    df_final = [pl.read_csv(file_path, columns=RAW_COLUMNS) for file_path in file_paths]

    # Align columns across all files
    columns = df_final[-1].columns
    df_final = pl.concat([data_df[columns] for data_df in df_final])

    # Filter by known users
    df_final = df_final.filter(df_final['user_id'].is_in(ALL_USERS))

    # Create globally unique session IDs
    df_final = (
        df_final.with_columns([
            pl.concat_str([
                pl.col('user_id').cast(pl.Utf8),
                pl.lit('_'),
                pl.col('session_id').cast(pl.Utf8)
            ]).alias('combined_key')
        ])
        .with_columns([
            pl.col('combined_key').rank(method='dense').cast(pl.Int32).alias('session_id')
        ])
        .drop('combined_key')
    )

    return df_final.to_pandas()


In [ ]:
class StatisticalFeatureDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return (
            torch.FloatTensor(self.features[idx]),
            torch.FloatTensor([self.labels[idx]]),
        )


class HdcStatisticalEncoder(torch.nn.Module):
    def __init__(self, input_size: int, out_dimension: int, dtype=torch.float32, device: str = "cpu"):
        super().__init__()
        self.input_size = input_size
        self.out_dimension = out_dimension
        self.device = device
        self.keys = embeddings.Random(input_size, out_dimension, dtype=dtype, device=device)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.to(self.device)  # shape: (batch_size, input_size)

        # Expand x to shape (batch_size, input_size, 1)
        x_expanded = x.unsqueeze(-1)  # shape: (B, I, 1)

        # Expand keys to shape (1, input_size, out_dimension)
        keys_expanded = self.keys.weight.unsqueeze(0)  # shape: (1, I, D)

        # Element-wise multiply: (B, I, 1) * (1, I, D) => (B, I, D)
        bound = x_expanded * keys_expanded

        # Multiset across input dimension
        hv = torchhd.multiset(bound)  # shape: (B, D)

        return torchhd.hard_quantize(hv)


class HdcModel(torch.nn.Module):
    def __init__(
        self,
        input_size: int,
        out_dimension: int,
        ngrams: int = 1,  # Not used in statistical encoder, but kept for config compatibility
        dtype=torch.float32,
        device: str = "cpu",
    ):
        super().__init__()
        self.encoder = HdcStatisticalEncoder(input_size, out_dimension, dtype=dtype, device=device)
        self.centroid = models.Centroid(out_dimension, NUM_TAC_LEVELS, dtype=dtype, device=device)
        self.device = device

    def add(self, x: torch.Tensor, y: torch.Tensor, lr: float):
        hv = self.encoder(x)
        labels = y.to(dtype=torch.int64)
        for i in range(len(hv)):
            self.centroid.add_adjust(hv[i].unsqueeze(0), labels[i], lr=lr)

    def adjust_reset(self):
        self.centroid.adjust_reset()

    def vector_norm(self, x, p=2, dim=None, keepdim=False):
        return torch.pow(torch.sum(torch.abs(x) ** p, dim=dim, keepdim=keepdim), 1 / p)

    def normalized_inference(self, input: torch.Tensor, dot: bool = False):
        weight = self.centroid.weight.detach().clone()
        norms = self.vector_norm(weight, p=2, dim=1, keepdim=True)
        norms.clamp_(min=1e-12)
        weight.div_(norms)
        return torchhd.functional.dot_similarity(input, weight) if dot else torchhd.functional.cosine_similarity(input, weight)

    def binary_hdc_output(self, outputs):
        return F.softmax(outputs, dim=1)[:, 1]

    def forward(self, x: torch.Tensor):
        hv = self.encoder(x)
        output = self.normalized_inference(hv, True)
        return output[:, 1]  # raw logits for class 1



def prepare_statistical_features(df, feature_columns):
    X = df[feature_columns].values
    y = df["tac_flg"].values
    return X, y


def validate_model(model, valid_loader, criterion, device):
    model.eval()
    val_preds = []
    val_labels = []
    val_loss = 0

    with torch.no_grad():
        for batch_X, batch_y in valid_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.squeeze(-1))
            val_loss += loss.item()

            val_preds.extend(outputs.cpu().numpy())
            val_labels.extend(batch_y.squeeze(-1).cpu().numpy())

    val_preds = np.array(val_preds)
    val_labels = np.array(val_labels)

    val_prauc = average_precision_score(val_labels, val_preds)
    val_rocauc = roc_auc_score(val_labels, val_preds)

    return {
        "loss": val_loss / len(valid_loader),
        "pr_auc": val_prauc,
        "roc_auc": val_rocauc,
    }


def train_model(
    model,
    train_loader,
    valid_loader,
    lr,
    device,
    epochs=100,
    patience=5,
):
    # Compute class weights (favoring minority class: drunk = 1)
    all_labels = torch.cat([y for _, y in train_loader], dim=0).squeeze()
    num_neg = (all_labels == 0).sum().item()  # sober
    num_pos = (all_labels == 1).sum().item()  # drunk
    pos_weight_val = num_neg / max(num_pos, 1)
    pos_weight = torch.tensor([pos_weight_val], device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)


    # Use weighted BCE loss
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    best_val_prauc = 0
    patience_counter = 0
    best_epoch = 0
    best_state = None

    for epoch in range(epochs):
        print(f"Epoch {epoch+1} / {epochs}")
        model.train()
        loss_total = 0
        for X, y in train_loader:
            model.add(X.to(device), y.to(device), lr)
            logits = model(X.to(device))  # raw logits
            loss = criterion(logits, y.to(device).squeeze(-1))
            loss_total += loss.item()

        model.adjust_reset()

        if valid_loader:
            val_preds, val_labels = inference_dataset(model, valid_loader, device)
            val_prauc = average_precision_score(val_labels, val_preds)

            if val_prauc >= best_val_prauc:
                best_val_prauc = val_prauc
                best_state = model.state_dict()
                best_epoch = epoch
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                break

    if best_state:
        model.load_state_dict(best_state)
    return model, None, best_epoch



def inference_dataset(model, data_loader, device, pred_threshold=None):
    model.eval()
    predictions = []
    labels = []
    with torch.no_grad():
        for batch_X, batch_y in data_loader:
            batch_X = batch_X.to(device)
            logits = model(batch_X)
            probs = torch.sigmoid(logits)
            predictions.extend(probs.cpu().numpy())
            labels.extend(batch_y.squeeze(-1).cpu().numpy())
    return np.array(predictions), np.array(labels)



def performance_calculation(pred_prob, gt_label, threshold=None):
    gt_label = gt_label.astype(int)
    if threshold is None:
        thresholds = np.linspace(0.01, 0.99, 99)
        best_f1 = 0
        best_threshold = 0.5
        for t in thresholds:
            temp_pred = (pred_prob >= t).astype(int)
            temp_f1 = f1_score(gt_label, temp_pred)
            if temp_f1 > best_f1:
                best_f1 = temp_f1
                best_threshold = t
        threshold = best_threshold

    pred_label = (pred_prob >= threshold).astype(int)
    return (
        roc_auc_score(gt_label, pred_prob),
        average_precision_score(gt_label, pred_prob),
        accuracy_score(gt_label, pred_label),
        accuracy_score(gt_label[gt_label == 0], pred_label[gt_label == 0]),
        accuracy_score(gt_label[gt_label == 1], pred_label[gt_label == 1]),
        f1_score(gt_label, pred_label),
        threshold
    )


def generate_configs(base_config):
    list_keys = [key for key, value in base_config.items() if isinstance(value, list)]
    if not list_keys:
        return [base_config]

    key = list_keys[0]
    values = base_config[key]
    configs = []

    for value in values:
        new_config = base_config.copy()
        new_config[key] = value
        configs.extend(generate_configs(new_config))

    return configs


In [41]:
def save_model_and_results(model, save_folder, train_preds, train_gt_labels, test_preds, test_gt_labels, metrics, config):
    """
    Save model, predictions, ground truth, metrics, and model structure to the specified folder.
    
    Args:
        model: The trained model
        save_folder: Folder path to save results
        train_preds: Training predictions
        train_gt_labels: Training ground truth labels
        test_preds: Test predictions
        test_gt_labels: Test ground truth labels
        metrics: Dictionary containing evaluation metrics
        config: Model configuration dictionary
    """
    # Create a timestamp for the save files
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save the model
    model_path = os.path.join(save_folder, f"model_{timestamp}.pt")
    torch.save(model.state_dict(), model_path)
    
    # Save model architecture as text
    model_structure_path = os.path.join(save_folder, f"model_structure_{timestamp}.txt")
    with open(model_structure_path, 'w') as f:
        f.write(str(model))
    
    # Save predictions and ground truth
    predictions_data = {
        'train_predictions': train_preds.tolist() if isinstance(train_preds, np.ndarray) else train_preds,
        'train_ground_truth': train_gt_labels.tolist() if isinstance(train_gt_labels, np.ndarray) else train_gt_labels,
        'test_predictions': test_preds.tolist() if isinstance(test_preds, np.ndarray) else test_preds,
        'test_ground_truth': test_gt_labels.tolist() if isinstance(test_gt_labels, np.ndarray) else test_gt_labels
    }
    pred_path = os.path.join(save_folder, f"predictions_{timestamp}.pkl")
    with open(pred_path, 'wb') as f:
        pickle.dump(predictions_data, f)
    
    # Save all metrics
    metrics_path = os.path.join(save_folder, f"metrics_{timestamp}.json")
    with open(metrics_path, 'w') as f:
        json.dump(metrics, f, indent=4)
    
    # Save the configuration
    config_path = os.path.join(save_folder, f"config_{timestamp}.json")
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=4)
    
    print(f"Model, predictions, ground truth, and metrics saved in {save_folder}")


def train_and_eval_final_model(best_config, best_threshold, df):
    print('\nBEGIN TRAIN AND EVALUATION FINAL MODEL\n')
    feature_columns = [
        "ZVALUEX_acc",
        "ZVALUEY_acc",
        "ZVALUEZ_acc",
        "ZVALUEX_gyro",
        "ZVALUEY_gyro",
        "ZVALUEZ_gyro",
        "ZHEARTRATE",
    ]

    # Hyper parameter loading
    device = best_config['device'] 
    window_size = best_config['window_size']
    batch_size = best_config['batch_size']
    hdc_dimension = best_config['hdc_dimension']
    ngrams = best_config['ngrams']
    learning_rate = best_config['learning_rate']
    epochs = best_config['epochs']
    patience = best_config['patience']
    runtime_log_fld = best_config['runtime_log_fld']
    overlap_ratio = best_config['overlap_ratio']
    
    train_user = list(set(ALL_USERS)- set(TEST_USERS))
    test_user = TEST_USERS
        
    train_data = df[df['user_id'].isin(train_user)]
    test_data = df[df['user_id'].isin(test_user)]


    # doing normalization (assume that we have the same scaler for all data, faster computing than do it separately)
    scaler = StandardScaler()
    train_data[STAT_FEATURE_COLUMNS] = scaler.fit_transform(train_data[STAT_FEATURE_COLUMNS])
    test_data[STAT_FEATURE_COLUMNS] = scaler.transform(test_data[STAT_FEATURE_COLUMNS])


    # Prepare statistical features
    feature_columns = [
        col for col in df.columns if col.startswith("ZVALUE") or col.startswith("ZHEARTRATE_")
    ]
    input_size = len(feature_columns)
    X_train = train_data[feature_columns].values
    y_train = train_data["tac_flg"].values
    X_test = test_data[feature_columns].values
    y_test = test_data["tac_flg"].values

    train_dataset = StatisticalFeatureDataset(X_train, y_train)
    test_dataset = StatisticalFeatureDataset(X_test, y_test)

    print(f"Users in train:{set(train_data['user_id'])}")
    print(f"Users in test:{set(test_data['user_id'])}")
    print(f"Number of windows for training:{len(X_train)}")
    print(f"Number of windows for testing:{len(X_test)}")

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    model = HdcModel(input_size, hdc_dimension, ngrams, device=device)

    criterion = nn.BCELoss()
    #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # 02. Train model (set patience to ensure that the model is trained for the best epoch)
    model, training_history, _ = train_model(
        model=model, train_loader=train_loader, valid_loader=None, lr=learning_rate, device=device, epochs=epochs
    )

    # 03. Inference
    train_preds, train_gt_labels = inference_dataset(model, train_loader, device)
    test_preds, test_gt_labels = inference_dataset(model, test_loader, device)

    # 04. Calculate performance of current config: ROC, PR-AUC, ACC, F1, Drunk ACC, Sober ACC
    train_roc_auc, train_pr_auc, train_accuracy, train_sober_acc, train_drunk_acc, train_f1, train_threshold = performance_calculation(train_preds, train_gt_labels, threshold=best_threshold)
    print(f"Training ROC-AUC: {train_roc_auc:.4f}, PR-AUC: {train_pr_auc:.4f}, Accuracy: {train_accuracy:.4f}, Sober Accuracy: {train_sober_acc:.4f}, Drunk Accuracy: {train_drunk_acc:.4f}, F1: {train_f1:.4f}, Threshold: {train_threshold:.4f}")
    test_roc_auc, test_pr_auc, test_accuracy, test_sober_acc, test_drunk_acc, test_f1, test_threshold = performance_calculation(test_preds, test_gt_labels, threshold=best_threshold)
    print(f"Test ROC-AUC: {test_roc_auc:.4f}, PR-AUC: {test_pr_auc:.4f}, Accuracy: {test_accuracy:.4f}, Sober Accuracy: {test_sober_acc:.4f}, Drunk Accuracy: {test_drunk_acc:.4f}, F1: {test_f1:.4f}, Threshold: {test_threshold:.4f}")    

    # 05. Save model, predictions, ground truth, metrics and model structure
    metrics = {
        'train': {
            'roc_auc': train_roc_auc,
            'pr_auc': train_pr_auc,
            'accuracy': train_accuracy,
            'sober_accuracy': train_sober_acc,
            'drunk_accuracy': train_drunk_acc,
            'f1': train_f1,
            'threshold': train_threshold
        },
        'test': {
            'roc_auc': test_roc_auc,
            'pr_auc': test_pr_auc,
            'accuracy': test_accuracy,
            'sober_accuracy': test_sober_acc,
            'drunk_accuracy': test_drunk_acc,
            'f1': test_f1,
            'threshold': test_threshold
        },
        'config': best_config
    }
    
    # Call the function to save all results
    save_model_and_results(
        model=model,
        save_folder=runtime_log_fld,
        train_preds=train_preds,
        train_gt_labels=train_gt_labels,
        test_preds=test_preds,
        test_gt_labels=test_gt_labels,
        metrics=metrics,
        config=best_config
    )
    
    # refresh GPU
    model.to("cpu")
    del model
    gc.collect()
    
    return train_accuracy, test_accuracy, metrics

def train_cross_validation(df, all_configs):
    print("="*50 + "\nBEGIN CROSSVALIDATION\n" + "="*50)    
    best_config = None 
    best_pr_auc = 0
    best_threshold = 0.5 

    for config_idx, config in enumerate(all_configs):
        print(f"\nCONFIG {config_idx}: {config}\n")

        feature_columns = [
            col for col in df.columns if col.startswith("ZVALUE") or col.startswith("ZHEARTRATE_")
        ]

        # Hyper parameter loading
        
        device = config['device'] 
        input_size = len(feature_columns)
        batch_size = config['batch_size']
        hdc_dimension = config['hdc_dimension']
        ngrams = config['ngrams']
        learning_rate = config['learning_rate']
        epochs = config['epochs']
        patience = config['patience']

        val_all_preds = []
        val_all_gt_labels = []
        val_trained_epoch = []

        for fold, (train_user, val_user) in enumerate(zip(TRAIN_USERS, VALID_USERS)):
            print("-" * 100)
            print('FOLD:', fold+1)
            print('TRAIN:', train_user)
            print('VAL:', val_user)

            train_data = df[df['user_id'].isin(train_user)].copy()
            val_data = df[df['user_id'].isin(val_user)].copy()

            # Prepare features and labels
            X_train = train_data[feature_columns].values
            y_train = train_data["tac_flg"].values
            X_val = val_data[feature_columns].values
            y_val = val_data["tac_flg"].values

            train_dataset = StatisticalFeatureDataset(X_train, y_train)
            val_dataset = StatisticalFeatureDataset(X_val, y_val)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size)

            model = HdcModel(input_size, hdc_dimension, ngrams, device=device)
            pos_weight = torch.tensor([len(y_train[y_train == 0]) / len(y_train[y_train == 1])]).to(device)
            pos_weight = torch.tensor([len(y_train[y_train == 0]) / len(y_train[y_train == 1])]).to(device)
            criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

            model, training_history, train_best_epoch = train_model(
                model, train_loader, val_loader, lr=learning_rate, device=device, 
                epochs=epochs, patience=patience
            )

            val_preds, val_gt_labels = inference_dataset(model, val_loader, device)
            val_all_preds.append(val_preds)
            val_all_gt_labels.append(val_gt_labels)
            val_trained_epoch.append(train_best_epoch)

            # refresh GPU
            model.to("cpu")
            del model
            gc.collect()

        # Evaluate performance
        val_all_preds = np.concatenate(val_all_preds)
        val_all_gt_labels = np.concatenate(val_all_gt_labels)
        val_roc_auc, val_pr_auc, val_accuracy, val_sober_acc, val_drunk_acc, val_f1, val_threshold = performance_calculation(
            val_all_preds, val_all_gt_labels
        )

        print(f"Validation ROC-AUC: {val_roc_auc:.4f}, PR-AUC: {val_pr_auc:.4f}, Accuracy: {val_accuracy:.4f}, Sober Accuracy: {val_sober_acc:.4f}, Drunk Accuracy: {val_drunk_acc:.4f}, F1: {val_f1:.4f}, Threshold: {val_threshold:.4f}")

        if val_pr_auc > best_pr_auc:
            best_pr_auc = val_pr_auc
            best_config = config
            best_threshold = val_threshold
            best_config_epoch = np.ceil(np.mean(val_trained_epoch)) + 1
            print(f"Updated best config: {best_config}, PR-AUC: {best_pr_auc:.4f}, Threshold: {best_threshold:.4f}, Epoch: {best_config_epoch:.2f}")

    print("="*50)
    best_config['epochs'] = int(best_config_epoch)
    best_config['patience'] = int(best_config_epoch)
    print(f'Final best config: {best_config}, Final best pr_auc: {best_pr_auc}, Final best threshold: {best_threshold}, Final best epoch: {best_config_epoch}')
    print("="*50 + "\nEND CROSSVALIDATION\n" + "="*50)    
    return best_config, best_pr_auc, best_threshold




In [42]:
def setup_logging(log_file_path):
    # Configure logging
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    
    # Remove existing handlers
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)
    
    # Add file handler
    file_handler = logging.FileHandler(log_file_path)
    file_handler.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    
    # Create a custom print function
    original_print = print
    
    def custom_print(*args, **kwargs):
        # Call original print
        # original_print(*args, **kwargs)
        # Log the printed content
        message = " ".join(str(arg) for arg in args)
        logger.info(f"PRINT: {message}")
    
    # Replace built-in print
    import builtins
    builtins.print = custom_print
    
    logger.info(f"Logging initialized to {os.path.abspath(log_file_path)}")
    return logger

In [43]:
def runtime():
    # 1. Set up logging
    runtime_log_fld = f"results/{MODEL_NAME}/{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    if not os.path.exists(runtime_log_fld):
        os.makedirs(runtime_log_fld)
    logger = setup_logging(f"{runtime_log_fld}/training.log")
    
    # 2. Set up configurations
    base_configs = BASE_CONFIGS
    base_configs['runtime_log_fld'] = runtime_log_fld   
    all_configs = generate_configs(base_configs)
    print(f"Total configurations: {len(all_configs)}")

    # 3. Load statistical feature data
    df = load_data()

    # 4. Train and evaluate all configurations (cross-validation)
    best_config, best_pr_auc, best_threshold = train_cross_validation(df, all_configs)

    # 5. Train and evaluate the final model with the best configuration
    train_and_eval_final_model(best_config, best_threshold, df)


In [44]:
runtime()